In [ ]:
# Install Gradio
!pip install gradio transformers -q

# Import the required Libraries
import gradio as gr
import numpy as np
import pandas as pd
import pickle
import transformers

from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from scipy.special import softmax

## App 1

In [ ]:
# Requirements
model_path = "KwameOO/covid-tweet-sentiment-analyzer-roberta"
tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = "@user" if t.startswith("@") and len(t) > 1 else t
        t = "http" if t.startswith("http") else t
        new_text.append(t)
    return " ".join(new_text)

# ---- Function to process the input and return prediction
def sentiment_analysis(text):
    text = preprocess(text)

    encoded_input = tokenizer(text, return_tensors = "pt") # for PyTorch-based models
    output = model(**encoded_input)
    scores_ = output[0][0].detach().numpy()
    scores_ = softmax(scores_)
    
    # Format output dict of scores
    labels = ["Negative", "Neutral", "Positive"]
    scores = {l:float(s) for (l,s) in zip(labels, scores_) }
    
    return scores


# ---- Gradio app interface
app = gr.Interface(fn = sentiment_analysis,
                   inputs = gr.Textbox("Write your text or tweet here..."),
                   outputs = "label",
                   title = "Sentiment Analysis of Tweets on COVID-19 Vaccines",
                   description  = "To vaccinate or not? This app analyzes sentiment of text based on tweets tweets about COVID-19 Vaccines using a fine-tuned roBERTA model",
                   interpretation = "default",
                   examples = [["The idea of a vaccine in record time sure sounds interesting!"]]
                   )

app.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## App 2

In [ ]:
# Load the fine-tuned model from the HF model hub
model = transformers.AutoModel.from_pretrained("KwameOO/covid-tweet-sentiment-analyzer-roberta")

# Define a function that takes in input and processes it with the model
def predict(inputs):
    input_ids = transformers.BertTokenizer.from_pretrained("KwameOO/covid-tweet-sentiment-analyzer-roberta").encode(inputs, return_tensors="pt")
    output = model(input_ids)[0]
    return output

# Create a Gradio interface for the model
app = gr.Interface(fn=predict,
                   inputs=gr.Textbox(prompt="Enter text here:"),
                   outputs=gr.Textbox(prompt= "Predicted sentiment:"),
                   title = "Sentiment Analysis of Tweets on COVID-19 Vaccines",
                   description  = "To vaccinate or not? This app analyzes sentiment of text based on tweets tweets about COVID-19 Vaccines using a fine-tuned roBERTA model"
                   )

# Launch the interface
app.launch()

Some weights of the model checkpoint at KwameOO/covid-tweet-sentiment-analyzer-roberta were not used when initializing RobertaModel: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at KwameOO/covid-tweet-sentiment-analyzer-roberta and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>